Load the NuGet package and print out the version to make sure we are using the latest and greatest

In [1]:
#r "nuget: ICSharpCode.Decompiler, 7.0.0.6372-preview3"

using System.Reflection.Metadata;
using ICSharpCode.Decompiler;
using ICSharpCode.Decompiler.CSharp;
using ICSharpCode.Decompiler.Metadata;
using ICSharpCode.Decompiler.TypeSystem;

Console.WriteLine(typeof(FullTypeName).Assembly.GetName());

ICSharpCode.Decompiler, Version=7.0.0.6372, Culture=neutral, PublicKeyToken=d4bfe873e7598c49


You must have compiled **frontends.sln** first (run “dotnet build” in ICSharpCode.Decompiler.PowerShell folder). Make sure to modify **basePath** to your repository path.

In [1]:
const string basePath = @"D:\GitWorkspace\ILSpy\";
string testAssemblyPath = basePath + @"ICSharpCode.Decompiler.PowerShell\bin\Release\netstandard2.0\ICSharpCode.Decompiler.dll";

var decompiler = new CSharpDecompiler(testAssemblyPath, new DecompilerSettings());

Get the count of types in this module

In [1]:
var types = decompiler.TypeSystem.MainModule.TypeDefinitions;
Console.WriteLine(types.Count());

1459


Decompile a known type (as a whole)

In [1]:
// ICSharpCode.Decompiler.Util.Empty<T> -> translates to `n, where n is the # of generic parameters
var nameOfGenericType = new FullTypeName("ICSharpCode.Decompiler.Util.Empty`1");
Console.WriteLine(decompiler.DecompileTypeAsString(nameOfGenericType));

using System;

namespace ICSharpCode.Decompiler.Util
{
	public static class Empty<T>
	{
		public static readonly T[] Array = System.Array.Empty<T>();
	}
}



If you want to decompile one single member (sample: first method)

In [1]:
var nameOfUniResolver =  new FullTypeName("ICSharpCode.Decompiler.Metadata.UniversalAssemblyResolver");
ITypeDefinition typeInfo = decompiler.TypeSystem.FindType(nameOfUniResolver).GetDefinition();
var tokenOfFirstMethod = typeInfo.Methods.First().MetadataToken;
Console.WriteLine(decompiler.DecompileAsString(tokenOfFirstMethod));

using System;

static UniversalAssemblyResolver()
{
	gac_paths = GetGacPaths();
	ZeroVersion = new Version(0, 0, 0, 0);
	if (Type.GetType("Mono.Runtime") != null)
	{
		decompilerRuntime = DecompilerRuntime.Mono;
	}
	else if (typeof(object).Assembly.GetName().Name == "System.Private.CoreLib")
	{
		decompilerRuntime = DecompilerRuntime.NETCoreApp;
	}
	else if (Environment.OSVersion.Platform == PlatformID.Unix)
	{
		decompilerRuntime = DecompilerRuntime.Mono;
	}
}



If you need access to low-level metadata tables

In [1]:
ITypeDefinition type = decompiler.TypeSystem.FindType(nameOfUniResolver).GetDefinition();
var module = type.ParentModule.PEFile;

Get the child namespaces

In [1]:
var icsdns = decompiler.TypeSystem.RootNamespace;
foreach (var cn in icsdns.ChildNamespaces) Console.WriteLine(cn.FullName);

Microsoft


System


LightJson


Humanizer


ICSharpCode


FxResources


Internal


Windows


MS


Get types in a single namespace

In [1]:
var typesInNamespace = icsdns.ChildNamespaces.First(cn => cn.FullName == "LightJson").Types;
Console.WriteLine(typesInNamespace.First().FullTypeName);

LightJson.JsonArray
